# HNU CE 프로그래밍언어론 (2023년 1학기) HW4
기말과제 파트1

- 이름: 윤승준
- 학번: 20180679

이름과 학번을 위에 작성하시오.


적절한 내용과 개수의 테스트를 작성하지 않으면 감점. 테스트가 적절하지 않거나 지나치게 부족하면 함수를 올바르게 작성했더라도 점수를 얻지 못할 수 있음에 유의!

----

In [1]:
:opt no-lint
{-# LANGUAGE ScopedTypeVariables FlexibleInstances TypeApplications RankNTypes KindSignatures #-}

----
## 문제 1. (20점) 9장 연습문제 4번

In [2]:
type Nm = String -- 변수 이름은 문자열로
data Expr = Var Nm         -- x
          | Lam Nm Expr    -- (λx.e)
          | App Expr Expr  -- (e1 e2)
          deriving (Eq, Ord)

type Addr = Int
type Env' = [(Nm,Addr)]
data Clos' = Cl' Expr Env' deriving (Eq,Ord)

lookupEnv' :: Nm -> Env' -> [Addr]
lookupEnv' x env = case lookup x env of
                     Nothing -> []
                     Just l -> [l]

import qualified Data.HashMap.Strict as M -- M.funcName으로 활용
import Data.HashMap.Strict (HashMap, (!), (!?)) -- 앞에 M. 없이
type Store = HashMap Int Clos'

lookupStore :: Addr -> Store -> [Clos']
lookupStore l st = case st !? l of
                     Nothing -> []
                     Just cl -> [cl]
                     
type Kont' = [Either Addr (Ctx,Env')]
data Ctx = Done | App1 Expr | App2 Expr  deriving (Eq,Ord)

In [3]:
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x e) = showParen (p > 1) $
      showString ("\\lambda "++x++".") . showsPrec 1 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
import IHaskell.Display (latex)
dTeX e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Expr where display = dTeX

instance Show Ctx where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ Done = showString "\\bullet "
  showsPrec p (App1 e2) = showParen (p > 9) $
      showString "\\bullet \\;" . showsPrec 10 e2
  showsPrec p (App2 e1) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\; \\bullet"
instance IHaskellDisplay Ctx where display = dTeX
                                               
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env' where
  show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Addr) where
  show (x,l) = x++"{\\mapsto}"++show l
instance Show Clos' where
  show (Cl' e env) = "\\langle "++show e++","++show env++"\\rangle "
instance {-# OVERLAPS #-} Show (Addr,Clos') where
  show (l,cl) = show l++"{\\mapsto}"++show cl
instance {-# OVERLAPS #-} Show Store where
  show s = "\\{"++ intercalate ",\\," (map show $ M.toList s) ++"\\}"

instance {-# OVERLAPS #-} IHaskellDisplay Env'  where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Addr) 
                                                where display = dTeX
instance IHaskellDisplay Clos'                  where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Addr,Clos')
                                                where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay Store where display = dTeX

In [4]:
stepCESKz :: (Clos', Store, Kont') -> [(Clos', Store, Kont')]
stepCESKz (Cl' (Var x) env, st, k) = [ (cl, st, Left l : k) |
                                         l <- lookupEnv' x env,
                                         cl <- lookupStore l st ]
stepCESKz (Cl' (App e1 e2) env, st, k) = [ (Cl' e1 env, st, Right (App1 e2, env) : k) ]
stepCESKz (Cl' (Lam x e) env1, st, Right (App1 e2, env) : k) = [ (Cl' e2 env, st, Right (App2 (Lam x e), env1): k) ]
stepCESKz (v2@(Cl' (Lam _ _) _), st, Right (App2 (Lam x e), env1) : k) = [ (Cl' e ((x, l) : env1), st', k) ]
  where
    l = length st
    st' = M.insert l (Cl' (Lam x e) env1) st
stepCESKz _ = []

In [5]:
hat f es = concat [f e | e <- es]
stepCESKz' = hat stepCESKz -- 여러 번 실행에 편리한 확장된 함수

In [6]:
-- 테스트 작성
sigma00 = M.empty :: Store
sigma01 = M.insert 1 (Cl' (Lam "x" $ Var "x") []) sigma00 :: Store
sigma02 = M.insert 2 (Cl' (Lam "z" $ Var "z") []) sigma01 :: Store
stepCESKz' [( Cl' (Var "y") [("y", 1)], sigma02, [Left 1, Right (Done, [])] )]
stepCESKz' it

stepCESKz' [( Cl' (App (Lam "x" $ Var "x") (Lam "y" $ Var "y")) [("x", 1)], sigma01, [Left 1, Right (Done, [])] )]
stepCESKz' it
stepCESKz' it
stepCESKz' it
stepCESKz' it

[(\langle \lambda x.x,\{\}\rangle ,\{1{\mapsto}\langle \lambda x.x,\{\}\rangle ,\,2{\mapsto}\langle \lambda z.z,\{\}\rangle \},[Left 1,Left 1,Right (\bullet ,\{\})])]

[]

[(\langle \lambda x.x,\{x{\mapsto}1\}\rangle ,\{1{\mapsto}\langle \lambda x.x,\{\}\rangle \},[Right (\bullet \;(\lambda y.y),\{x{\mapsto}1\}),Left 1,Right (\bullet ,\{\})])]

[(\langle \lambda y.y,\{x{\mapsto}1\}\rangle ,\{1{\mapsto}\langle \lambda x.x,\{\}\rangle \},[Right ((\lambda x.x)\; \bullet,\{x{\mapsto}1\}),Left 1,Right (\bullet ,\{\})])]

[(\langle x,\{x{\mapsto}1,\,x{\mapsto}1\}\rangle ,\{1{\mapsto}\langle \lambda x.x,\{x{\mapsto}1\}\rangle \},[Left 1,Right (\bullet ,\{\})])]

[(\langle \lambda x.x,\{x{\mapsto}1\}\rangle ,\{1{\mapsto}\langle \lambda x.x,\{x{\mapsto}1\}\rangle \},[Left 1,Left 1,Right (\bullet ,\{\})])]

[]

----
## 문제 2. (10점) 11장 연습문제 3번

교재 11장에서 필요한 정의를 가져와서 활용하라.

In [7]:
type Parser tok a = [tok] -> [(a, [tok])]

In [8]:
return :: a -> Parser tok a -- 입력 토큰열 ts를 소비하지 않고
return v = \ts -> [(v,ts)]  -- 그냥 v를 리턴하며 성공하는 파서

failure :: Parser tok a -- 입력에 관계없이 무조건 실패하는 파서
failure = \_ -> []

In [9]:
item :: Parser tok tok
item []     = []       -- 길이 0인 토큰열에 대해서는 실패
item (t:ts) = [(t,ts)] -- 맨 앞의 토큰 t하나만 처리해 t를 리턴 

In [10]:
(>>=) :: Parser tok a -> (a -> Parser tok b) -> Parser tok b
p1 >>= pf = \ts -> [ (v2,ts2) | (v1,ts1) <- p1 ts,
{- 이어붙이기 sequencing -}       (v2,ts2) <- (pf v1) ts1 ]

In [11]:
sat :: (tok -> Bool) -> Parser tok tok
sat test = item >>= \t ->          -- 토큰 하나를 읽어들여
           if test t then return t -- 조건에 맞는 경우에만 성공하고 
                     else failure  -- 그렇지 않으면 실패하는 파서

In [12]:
(<|>) :: Parser tok a -> Parser tok a -> Parser tok a  -- 선택 choice
p1 <|> p2 = \ts -> case p1 ts of
                     []  -> p2 ts  -- 첫번째 파서가 실패하면 두번째로
                     rs1 -> rs1    -- 첫번째가 성공하면 첫번째만으로

In [13]:
many, many1  :: Parser tok a -> Parser tok [a]  -- 상호재귀적으로 정의됨
many  p = many1 p <|> return [] -- 1회 성공 또는 무조건 0회라 치고 성공

many1 p = p      >>= \v  ->  -- 한번 성공한 다음
          many p >>= \vs ->  -- 0회 이상 성공
          return (v:vs)

In [14]:
import Data.Char ( isDigit, isLower, isUpper,
                   isAlpha, isAlphaNum, isSpace )
digit = sat isDigit
lower = sat isLower
upper = sat isUpper
letter = sat isAlpha
alphanum = sat isAlphaNum
space = sat isSpace

In [15]:
char :: Char -> Parser Char Char    -- char c는 주어진 글자 c와 첫글자가
char c = sat (==c)                  -- 일치하는 경우에만 성공하는 파서

string :: String -> Parser Char String  -- string s는 주어진 문자열 s와
string []     = return []               -- 앞부분이 일치하는 경우에만 성공
string (c:cs) = char c    >>= \_ ->
                string cs >>= \_ ->
                return (c:cs)

In [16]:
nat :: Parser Char Int
nat = many1 digit >>= \s ->  -- 1개 이상의 digit(숫자)
      return (read s)        -- 문자열 s를 정수로 변환한 값으로 성공시킴

ident :: Parser Char String
ident = lower         >>= \c  ->  -- 첫글자는 소문자
        many alphanum >>= \cs ->  -- 그 뒤에는 0개 이상의 문자 또는 숫자
        return (c:cs)

spaces  :: Parser Char ()
spaces  = many  space >>= \_ -> return ()  -- 0개 이상의 공백문자 처리
spaces1 :: Parser Char ()
spaces1 = many1 space >>= \_ -> return ()  -- 1개 이상의 공백문자 처리

In [17]:
data Tok = KW String -- 키워드
         | ID String -- 변수 이름
         | INT Int   -- 정수
         | LP        -- (
         | RP        -- )
         | LAM       -- \
         | DOT       -- .
         | ADD       -- +
         deriving (Eq,Ord,Show)

In [18]:
word :: Parser Char Tok
word = ident  >>= \s ->
       if s `elem` ["if","then","else"] then return (KW s)
                                        else return (ID s)

natural :: Parser Char Tok
natural = nat >>= \n -> return (INT n)

lp :: Parser Char Tok
lp = char '(' >>= \c -> return LP

rp :: Parser Char Tok
rp = char ')' >>= \c -> return RP

lam :: Parser Char Tok
lam = char '\\' >>= \c -> return LAM

dot :: Parser Char Tok
dot = char '.' >>= \c -> return DOT

add :: Parser Char Tok
add = char '+' >>= \c -> return ADD

In [19]:
tok :: Parser Char a -> Parser Char a
tok p = p      >>= \v ->
        spaces >>= \_ ->
        return v
        
lexFAC = (many (tok word <|> tok natural <|> tok lp <|> tok rp <|> tok lam <|> tok dot <|> tok add))

In [20]:
lexFAC "if b1 then 123 else x3  "
lexFAC "if b1 then 123 else 3 + (\\y. y) "

[([KW "if",ID "b1",KW "then",INT 123,KW "else",ID "x3"],"")]

[([KW "if",ID "b1",KW "then",INT 123,KW "else",INT 3,ADD,LP,LAM,ID "y",DOT,ID "y",RP],"")]

----
## 문제 3. (15점) 12장 연습문제 3번

모든 타입(Ty)을 원소로 나열하는 무한 리스트 `allTy`를 정의해 보라.
원소가 리스트에 나타나는 순서는 `Z`를 1개만 포하는 타입, `Z`를 2개 포함하는 타입들, `Z`를 3개 포함하는 타입들, ... 이런 순서로 나타나도록 정의하라.

그러니까 `allTy`는
$[\,\mathbb{Z},
  \;\mathbb{Z}{\to}\mathbb{Z},
  \;\mathbb{Z}{\to}\mathbb{Z}{\to}\mathbb{Z},
  \;(\mathbb{Z}{\to}\mathbb{Z}){\to}\mathbb{Z},
  \;\ldots]$와 같은 리스트이다.

In [21]:
data Ty = Z | Arr Ty Ty  deriving (Eq,Ord, Show)

In [22]:
orTy 1 = [Z]
orTy 2 = [Arr Z Z]
orTy n = [Arr Z t | t <- orTy (n-1)] ++ [Arr t Z | t <- orTy (n-1)]

allTy = [t | n <- [1, 2..], t <- orTy n] -- 필요하다면 여러 개의 등식이나 다른 보조 함수를 정의해서 활용해도 좋다

In [23]:
-- 테스트로 앞에서부터 적절한 개수의 타입들을 끊어서 보기
take 30 allTy

[Z,Arr Z Z,Arr Z (Arr Z Z),Arr (Arr Z Z) Z,Arr Z (Arr Z (Arr Z Z)),Arr Z (Arr (Arr Z Z) Z),Arr (Arr Z (Arr Z Z)) Z,Arr (Arr (Arr Z Z) Z) Z,Arr Z (Arr Z (Arr Z (Arr Z Z))),Arr Z (Arr Z (Arr (Arr Z Z) Z)),Arr Z (Arr (Arr Z (Arr Z Z)) Z),Arr Z (Arr (Arr (Arr Z Z) Z) Z),Arr (Arr Z (Arr Z (Arr Z Z))) Z,Arr (Arr Z (Arr (Arr Z Z) Z)) Z,Arr (Arr (Arr Z (Arr Z Z)) Z) Z,Arr (Arr (Arr (Arr Z Z) Z) Z) Z,Arr Z (Arr Z (Arr Z (Arr Z (Arr Z Z)))),Arr Z (Arr Z (Arr Z (Arr (Arr Z Z) Z))),Arr Z (Arr Z (Arr (Arr Z (Arr Z Z)) Z)),Arr Z (Arr Z (Arr (Arr (Arr Z Z) Z) Z)),Arr Z (Arr (Arr Z (Arr Z (Arr Z Z))) Z),Arr Z (Arr (Arr Z (Arr (Arr Z Z) Z)) Z),Arr Z (Arr (Arr (Arr Z (Arr Z Z)) Z) Z),Arr Z (Arr (Arr (Arr (Arr Z Z) Z) Z) Z),Arr (Arr Z (Arr Z (Arr Z (Arr Z Z)))) Z,Arr (Arr Z (Arr Z (Arr (Arr Z Z) Z))) Z,Arr (Arr Z (Arr (Arr Z (Arr Z Z)) Z)) Z,Arr (Arr Z (Arr (Arr (Arr Z Z) Z) Z)) Z,Arr (Arr (Arr Z (Arr Z (Arr Z Z))) Z) Z,Arr (Arr (Arr Z (Arr (Arr Z Z) Z)) Z) Z]

## 문제 4. (15점) 12장 연습문제 1번. `eval` 함수만 완성하라. `ev`는 작성하지 않아도 됨
재귀함수 관련 `undefined`로 남겨진 부분을 채워넣고 재귀함수 실행을 테스트해 보라

In [24]:
type Nm = String  -- 변수 이름은 문자열로
data Ty = Z | Arr Ty Ty  deriving (Eq,Ord)
data Expr = Var Nm             -- x
          | Lam Nm Ty Expr     -- (\x:t.e)
          | Rec Nm Ty Expr     -- rec f:t (\x:t2.e)
          | App Expr Expr      -- (e1 e2)
          | Lit Int            -- n
          | Add Expr Expr      -- e1 + e2
          | If Expr Expr Expr  -- if e then e1 else e0
          deriving (Eq, Ord)

data Val = Cl Expr Env | Vi Int  deriving (Eq,Ord)
type Env = [ (Nm, Val) ]
type TEnv = [ (Nm, Ty) ] -- 타입환경

In [25]:
instance Show Ty where
  showsPrec _ Z = showString "\\mathbb{Z}"
  showsPrec p (Arr t1 t2) = showParen (p > 1) $
      showsPrec p t1 . showString "{\\to}" . showsPrec p t2
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x t e) = showParen (p > 1) $
      showString ("\\lambda "++x++"{:}"++show t++".") . showsPrec 1 e
  showsPrec p (Rec f t e) = showParen (p > 9) $
      showString ("\\textbf{rec}\\,"++f++"{:}") . showsPrec 0 t .
      showString "\\, " . showsPrec 2 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
  showsPrec _ (Lit n) = shows n
  showsPrec p (Add e1 e2) = showParen (p > 6) $
      showsPrec 6 e1 . showString "+" . showsPrec 7 e2
  showsPrec p (If e e1 e0) = showParen (p > 2) $
      showString "\\textbf{if}\\;" . showsPrec 3 e .
      showString "\\;\\textbf{then}\\;" . showsPrec 3 e1 .
      showString "\\;\\textbf{else}\\;" . showsPrec 3 e0

import IHaskell.Display (latex)
dTeX e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Expr  where display = dTeX
instance IHaskellDisplay Ty    where display = dTeX

In [26]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show TEnv where
    show tenv = "\\{"++ intercalate ",\\," (map show tenv) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Val) where
    show (x,cl) = x++"{\\mapsto}"++show cl
instance {-# OVERLAPS #-} Show (Nm,Ty) where
    show (x,t) = x++"{\\mapsto}"++show t
instance Show Val where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "
    show (Vi n) = " "++show n++" "
instance {-# OVERLAPS #-} IHaskellDisplay Env    where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay TEnv   where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Val) where display=dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Ty)  where display=dTeX
instance IHaskellDisplay Val                     where display = dTeX

In [27]:
eval :: Env -> Expr -> Val
eval env (Var x) =
    case lookup x env of
        Nothing -> error (x ++ " not defined")
        Just v -> v
eval env e@(Lam _ _ _) = Cl e env
eval env (Rec f t e@(Lam _ _ _)) =
    let closure = Cl e env'
        env' = (f, closure) : env
    in closure
eval env (App e1 e2) =
    case v1 of
        Cl (Lam x _ e) env1 -> eval ((x, v2) : env1) e
        _ -> error (show v1 ++ " not closure")
    where
        v1 = eval env e1
        v2 = eval env e2
eval _ (Lit n) = Vi n
eval env (Add e1 e2) =
    case (v1, v2) of
        (Vi n1, Vi n2) -> Vi (n1 + n2)
        (Vi _, _) -> error (show v2 ++ " not int")
        (_, Vi _) -> error (show v1 ++ " not int")
    where
        v1 = eval env e1
        v2 = eval env e2
eval env (If e e1 e0) =
    case eval env e of
        Vi 0 -> eval env e0
        Vi _ -> eval env e1
        _ -> error (show e ++ " not int")


In [28]:
e1 = Lam "x" Z . Lam "y" Z $ If (Var "x") (Var "x" `Add` Var "y")
                                          (Var "y" `Add` Var "y")
e2 = Lit 2
e3 = Lit 3

eval [] e1
eval [] e2

In [29]:
App e1 e2
eval [] (App e1 e2)

In [30]:
App (App e1 e2) e3
eval [] (App (App e1 e2) e3)

In [31]:
App (App e1 (Lit 0)) e3
eval [] (App (App e1 (Lit 0)) e3)

In [32]:
e4 = Rec "f" (Arr Z Z) .
         Lam "i" Z $ If i (i `Add` App f (i `Add` Lit (-1))) i
    where f = Var "f"
          i = Var "i"
e4 -- rec f (\i.if i then i + f(i + -1) else i)

In [33]:
-- rec f (\i.if i then i + f(i + -1) else i) 100
eval [] (e4 `App` Lit 100)  -- 100부터 0까지의 총합

## 문제 5. (30점) 12장 연습문제 4번,5번,6번

In [34]:
-- 4번
check tenv e t = infer tenv e == [t]  -- check는 infer로 간단히 정의 가능

infer :: TEnv -> Expr -> [Ty]  -- 적절한 타입이 존재하지 않으면 빈 리스트
infer tenv (Var x)      = case lookup x tenv of Just t  -> [t]
                                                Nothing -> []
infer tenv (Lam x t2 e) = case infer ((x, t2) : tenv) e of
  [t] -> [Arr t2 t]
  _   -> []
infer tenv (Rec f t e) = case infer ((f, t) : tenv) e of
  [t'] | t == t' -> [t]
  _              -> []
infer tenv (App e1 e2) = case infer tenv e1 of
  [Arr t2 t] -> if check tenv e2 t2 then [t] else []
  _          -> []
infer tenv (Lit _) = [Z]
infer tenv (Add e1 e2) = if check tenv e1 Z && check tenv e2 Z then [Z] else []
infer tenv (If e e1 e0) = case infer tenv e of
  [Z] -> if infer tenv e1 == infer tenv e0 then infer tenv e1 else []
  _   -> []

In [35]:
-- 5번
tenv :: TEnv
tenv = [("x", Z)]

infer tenv $ Var "x"  -- 자유변수 x
-- 타입 추론 가능 
infer tenv $ Lit 3 `App` Lit 1  -- 3을 1에 함수처럼 적용
-- 불가 
infer tenv $ Lit 3 `Add` (Lam "x" Z $ Var "x")  -- 3과 함수요약식의 덧셈
-- 불가
infer tenv $ If (Lit 0) (Lam "x" Z $ Var "x") (Lit 3)
-- 불가

In [36]:
-- 6번
infer tenv e4